In [1]:
import csv
import numpy as np
import emoji
import matplotlib.pyplot as plt

%matplotlib inline

Emoji Class

In [2]:
emoji_dictionary = {"0": ":heart_with_arrow:",
                    "1": ":baseball:",
                    "2": ":smile:",
                    "3": ":disappointed:",
                    "4": ":fork_and_knife:"}

def label_to_emoji(label):
    """
    Converts a label (int or string) into the corresponding emoji code (string) ready to be printed
    """
    return emoji.emojize(emoji_dictionary[str(label)], use_aliases=True)

<img src="images/data_set.png" style="width:700px;height:300px;">

Load Dataset

In [3]:
def read_csv(filename):
    Phrase = []
    Emoji = []
    with open (filename) as f:
        data = csv.reader(f)
        for row in data:
            Phrase.append(row[0])
            Emoji.append(row[1])
    X = np.asarray(Phrase)
    Y = np.asarray(Emoji,dtype=int)
    return X,Y
X_train, Y_train = read_csv('data/train_emoji.csv')
X_test, Y_test = read_csv('data/tesss.csv')
### Example
index = 10
print X_train[index],label_to_emoji(Y_train[index])
###

she did not answer my text  😞


Get Max Length

In [4]:
max_len = len(max(X_train, key=len).split())
print 'max length of sentences is ', max_len

max length of sentences is  10


One-hot convert

In [5]:
y_train = np.eye(5)[Y_train.reshape(-1)]
y_test = np.eye(5)[Y_test.reshape(-1)]
### Example
index = 50
print Y_train[index], 'is converted into one hot', y_train[index]
###

0 is converted into one hot [1. 0. 0. 0. 0.]


Load Word Vectors

In [6]:
def read_GloVe_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map
word_to_index, index_to_word, word_to_vec_map = read_GloVe_vecs('data/glove.6B.50d.txt')

Average words meaning in a sentence. Can be used for naive Neural Network

In [7]:
# def meaning_encoding(sentence, word_to_vec_map):
#     words = sentence.lower().split()
#     meaning_code = np.zeros(word_to_vec_map[words[0]].shape)
#     for word in words:
#         meaning_code += word_to_vec_map[word]
#     avg = meaning_code/len(words)
#     return avg
# #### Example
# avg = meaning_encoding("Morrocan couscous is my favorite dish", word_to_vec_map)
# print("avg = ", avg)
# ####

Advanced LSTM Model in Keras

In [8]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)

/Users/fengjinzhao/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


<img src="images/embedding1.png" style="width:700px;height:250px;">

In [9]:
def to_indices(X,word_to_index,max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    m = X.shape[0]
    X_indices = np.zeros((m,max_len))
    for i in range(m):
        words = X[i].lower().split()
        j = 0 
        for word in words:
            X_indices[i,j] = word_to_index[word]
            j += 1
    return X_indices
### Example
X1 = np.array(["funny lol", "lets play baseball", "food is ready for you"])
X1_indices = to_indices(X1,word_to_index, max_len = 5)
print("X1 =", X1)
print("X1_indices =", X1_indices)   
###

('X1 =', array(['funny lol', 'lets play baseball', 'food is ready for you'],
      dtype='|S21'))
('X1_indices =', array([[155345., 225122.,      0.,      0.,      0.],
       [220930., 286375.,  69714.,      0.,      0.],
       [151204., 192973., 302254., 151349., 394475.]]))


In [10]:
def pretrained_embedding_layer(word_to_vec_map,word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    dim = word_to_vec_map['cucumber'].shape[0]
    ebd_matrix = np.zeros((vocab_len,dim))
    for word,index in word_to_index.items():
        ebd_matrix[index,:] = word_to_vec_map[word]
        
    # Define Keras embedding layer with the correct output/input sizes    
    Embedding_layer = Embedding(vocab_len,dim,trainable=False)
    # Build the embedding layer, it is required before setting the weights of the embedding layer.
    Embedding_layer.build((None,))
    # Set the weights of the embedding layer to the embedding matrix.
    Embedding_layer.set_weights([ebd_matrix])
    
    return Embedding_layer
### Example
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])
###

('weights[0][1][3] =', -0.3403)


<img src="images/emojifier-v2.png" style="width:700px;height:400px;"> <br>

In [11]:
def Emojify(input_shape,word_to_vec_map,word_to_index):
    """
    Function creating the Emojify model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    sentence_indices = Input(shape=input_shape,dtype='int32')
    
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    embeddings = embedding_layer(sentence_indices) 
    
    X = LSTM(128,return_sequences=True)(embeddings)
    X = Dropout(0.5)(X)
    X = LSTM(128,return_sequences=False)(X)
    X = Dropout(0.5)(X)
    X = Dense(5)(X)
    X = Activation('softmax')(X)
    
    model = Model(inputs=sentence_indices,outputs=X)
    return model

In [12]:
model = Emojify((max_len,), word_to_vec_map, word_to_index)
model.summary()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 10, 50)            20000050  
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 128)           91648     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
den

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [14]:
X_train_indices = to_indices(X_train,word_to_index,max_len)
model.fit(X_train_indices, y_train, epochs = 100, batch_size = 32, shuffle=True)

Epoch 1/100
132/132 [==============================] - 1s - loss: 1.6083 - acc: 0.1970     
Epoch 2/100
132/132 [==============================] - 0s - loss: 1.5322 - acc: 0.2955     
Epoch 3/100
132/132 [==============================] - 0s - loss: 1.5010 - acc: 0.3258     
Epoch 4/100
132/132 [==============================] - 0s - loss: 1.4386 - acc: 0.3561     
Epoch 5/100
132/132 [==============================] - 0s - loss: 1.3471 - acc: 0.4545     
Epoch 6/100
132/132 [==============================] - 0s - loss: 1.2332 - acc: 0.5076     
Epoch 7/100
132/132 [==============================] - 0s - loss: 1.1761 - acc: 0.4470     
Epoch 8/100
132/132 [==============================] - 0s - loss: 1.0540 - acc: 0.5758     
Epoch 9/100
132/132 [==============================] - 0s - loss: 0.8765 - acc: 0.7121     
Epoch 10/100
132/132 [==============================] - 0s - loss: 0.8228 - acc: 0.6970     
Epoch 11/100
132/132 [==============================] - 0s - loss: 0.7025 - acc

132/132 [==============================] - 0s - loss: 0.0016 - acc: 1.0000     
Epoch 90/100
132/132 [==============================] - 0s - loss: 0.0018 - acc: 1.0000     
Epoch 91/100
132/132 [==============================] - 0s - loss: 0.0015 - acc: 1.0000     
Epoch 92/100
132/132 [==============================] - 0s - loss: 0.0015 - acc: 1.0000     
Epoch 93/100
132/132 [==============================] - 0s - loss: 0.0015 - acc: 1.0000     
Epoch 94/100
132/132 [==============================] - 0s - loss: 0.0016 - acc: 1.0000     
Epoch 95/100
132/132 [==============================] - 0s - loss: 0.0016 - acc: 1.0000        
Epoch 96/100
132/132 [==============================] - 0s - loss: 0.0014 - acc: 1.0000     
Epoch 97/100
132/132 [==============================] - 0s - loss: 0.0016 - acc: 1.0000     
Epoch 98/100
132/132 [==============================] - 0s - loss: 0.0012 - acc: 1.0000        
Epoch 99/100
132/132 [==============================] - 0s - loss: 0.0012 - a

In [15]:
X_test_indices = to_indices(X_test,word_to_index,max_len)
loss, acc = model.evaluate(X_test_indices, y_test)
print()
print("Test accuracy = ", acc)

32/56 [================>.............] - ETA: 0s()
('Test accuracy = ', 0.8750000085149493)


In [16]:
# This code allows you to see the mislabelled examples
pred = model.predict(X_test_indices)
for i in range(len(X_test)):
    x = X_test_indices
    num = np.argmax(pred[i])
    if(num != Y_test[i]):
        print('Expected emoji:'+ label_to_emoji(Y_test[i]) + ' prediction: '+ X_test[i] + label_to_emoji(num).strip())

Expected emoji:😞 prediction: work is hard	😄
Expected emoji:😞 prediction: This girl is messing with me	💘
Expected emoji:💘 prediction: I love taking breaks	😞
Expected emoji:😄 prediction: you brighten my day	💘
Expected emoji:😞 prediction: she is a bully	💘
Expected emoji:😞 prediction: go away	🍴
Expected emoji:🍴 prediction: I did not have breakfast 💘


In [17]:
test = np.array(['not feeling happy'])
test_indices = to_indices(test, word_to_index, max_len)
print(test[0] +' '+  label_to_emoji(np.argmax(model.predict(test_indices))))

not feeling happy 😞
